In [17]:
import json
import datetime
from flask import Flask, jsonify
import hashlib
import os

In [18]:
class BitCoinBlockChain:
    def __init__(self):
        self.chain = []
        self.create_block(proof = 1, prev = "0")
    def create_block(self, proof, prev):
        block = {
            "index": len(self.chain) + 1,
            "timestamp": str(datetime.datetime.now()), 
            "proof": proof,
            "prev": prev
        }
        self.chain.append(block)
        return block
    
    def get_last_block(self):
        return self.chain[-1]
    
    def proofOfWork(self, previous_proof):
        new_proof = 1
        solved = False
        while solved is False:
            hashProblem = hashlib.sha256(str(new_proof ** 2 - previous_proof ** 2).encode()).hexdigest()
            if hashProblem[:4] == "0000":
                solved = True
            else:
                new_proof += 1
        return new_proof
    
    def hashBlock(self, block):
        encoded_bock = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_bock).hexdigest()
    
    def isChainValid(self, chain):
        previousBlock = chain[0]
        current_index = 1
        
        while current_index < len(chain):
            block = chain[current_index]
            if block["prev"] != self.hashBlock(previousBlock):
                return False
            currentProof = block["proof"]
            previous_proof = previousBlock["proof"]
            hashProblem = hashlib.sha256(str(currentProof ** 2 - previous_proof ** 2).encode()).hexdigest()
            if hashProblem[:4] != "0000":
                return False
        return True

In [19]:
app = Flask(__name__)

In [20]:
BTH = BitCoinBlockChain()

In [21]:
@app.route("/mine", methods = ["GET"])
def mine():
    prev_block = BTH.get_last_block()
    prev_proof = prev_block["proof"]
    proof = BTH.proofOfWork(prev_proof)
    previousHash = BTH.hashBlock(prev_block)
    block = BTH.create_block(proof, previousHash)
    response = {
        "success": "You have successfully mined a block",
        "timestamp": block["timestamp"],
        "index": block["index"],
        "proof": block["proof"],
        "prev": block["prev"]
    }
    return jsonify(response), 200

In [22]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": BTH.chain,
        "length": len(BTH.chain)
    }
    return jsonify(response), 200

In [23]:
if __name__=="__main__":
    app.run(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 4444)))

 * Running on http://0.0.0.0:4444/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Dec/2021 16:43:32] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:43:44] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:43:46] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:43:46] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:43:47] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:43:48] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:43:49] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:43:52] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:49:16] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:50:00] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:50:01] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:50:02] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:50:02] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:50:04] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2021 16:50:06] "GET /